In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('sample_data/heart.csv')

In [ ]:
data.head(1)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.0,1.9,130,1,0,4,1


In [ ]:
processed_data = pd.DataFrame()

In [ ]:
processed_data['age'] = data['age'].apply(lambda age : 'valid' if (1 < age and age <= 75) else 'invalid')
processed_data['anaemia'] = data['anaemia'].apply(lambda disease : 'present' if disease == 1 else 'absent')
processed_data['diabetes'] = data['diabetes'].apply(lambda disease : 'present' if disease == 1 else 'absent')
processed_data['high_blood_pressure'] = data['high_blood_pressure'].apply(lambda disease : 'present' if disease == 1 else 'absent')
processed_data['sex'] = data['sex'].apply(lambda gender : 'male' if gender == 1 else 'female')
processed_data['smoking'] = data['smoking'].apply(lambda person : 'smoker' if person == 1 else 'nonSmoker')
processed_data['DEATH_EVENT'] = data['DEATH_EVENT'].apply(lambda event : 'yes' if event == 1 else 'no')

In [ ]:
trainingData = processed_data[:50]
testingData = processed_data[50:]

In [ ]:
from copy import deepcopy

In [ ]:
# defining node in the decision tree
class Node:
  def __init__(self, name = '', parent = None, children = {}, classficationResult = False, leafNode = False):
    self.name = deepcopy(name)
    self.parent = parent
    self.children = deepcopy(children)
    self.classficationResult = classficationResult

In [ ]:
def calGiniImpurity(yesCnt, noCnt):
  total = yesCnt+noCnt
  pYes = yesCnt/total
  pNo = noCnt/total
  impurity = 1 - (pYes*pYes) - (pNo*pNo)
  return impurity

In [ ]:
def findFeatureMetaData(featureData, output):

  featureValueMetaData = {}

  for featureValue, outputVal in zip(featureData, output):
    if featureValue in featureValueMetaData:
      featureValueMetaData[featureValue]["cnt"] += 1
      featureValueMetaData[featureValue]["yes"] += (1 if outputVal == 'yes' else 0)
      featureValueMetaData[featureValue]["no"] += (1 if outputVal == 'no' else 0)

    else:
      featureValueMetaData[featureValue] = {
          "cnt" : 1,
          "yes" : (1 if outputVal == 'yes' else 0),
          "no" : (1 if outputVal == 'no' else 0)
      }
    
  return featureValueMetaData

In [ ]:
def calWeightedAvgGiniImpurity(featureMetaData):
  total = 0

  weightedAvgGiniImpurity = 0

  for k, v in featureMetaData.items():
    giniImpurity = calGiniImpurity(v['yes'], v['no'])
    weightedAvgGiniImpurity += giniImpurity * v['cnt']

    total += v['cnt']
  
  weightedAvgGiniImpurity /= total

  return weightedAvgGiniImpurity

In [ ]:
def buildTree(trainingData, root):

  dataCols = [col for col in trainingData.columns if col != 'DEATH_EVENT']

  if len(dataCols) == 0:
    return

  data = trainingData[dataCols]
  output = trainingData['DEATH_EVENT']

  totalYesCnt = 0
  totalNoCnt = 0

  for outputVal in output:
    if outputVal == 'yes':
      totalYesCnt += 1
    else:
      totalNoCnt += 1

  totalImpurity = calGiniImpurity(totalYesCnt, totalNoCnt)

  minGiniImpurity = totalImpurity
  bestFeatureName = None
  bestFeatureMetaData = None

  # 1) cal gini impurity for each column
  # 2) cal gini impurity for each distinct attribute value in 
  #    a col and then take the weighted average to calculate the gini 
  #    impurity for the whole col
  for col in data.columns:
    featureMetaData = findFeatureMetaData(data[col], output)
    weightedAvgGiniImpurity = calWeightedAvgGiniImpurity(featureMetaData)
    if weightedAvgGiniImpurity < minGiniImpurity:
      minGiniImpurity = weightedAvgGiniImpurity
      bestFeatureName = col
      bestFeatureMetaData = featureMetaData

  if bestFeatureName == None:
    # make it a leaf node with proper classification
    yesSeries = trainingData[output == 'yes']['DEATH_EVENT'].value_counts()
    noSeries = trainingData[output == 'no']['DEATH_EVENT'].value_counts()
    yesCnt = yesSeries['yes'] if len(yesSeries) > 0 else 0
    noCnt = noSeries['no'] if len(noSeries) > 0 else 0
    root.classficationResult = (yesCnt >= noCnt)
  else:
    root.name = bestFeatureName
    for featureValue, metaData in bestFeatureMetaData.items():
      reducedTrainingData = trainingData[trainingData[bestFeatureName] == featureValue]
      reducedTrainingData = reducedTrainingData.loc[:, trainingData.columns != bestFeatureName]
      root.children[featureValue] = Node(parent=root)
      buildTree(reducedTrainingData, root.children[featureValue])

In [ ]:
root = Node()
buildTree(trainingData, root)

In [ ]:
def printDecisionTree(root, level):
  if root == None:
    return
  print(level, root.name, root.children.keys())
  for k, v in root.children.items():
    printDecisionTree(v, level+1)

In [ ]:
level = 0
printDecisionTree(root, level)

0 smoking dict_keys(['nonSmoker', 'smoker'])
1 age dict_keys(['valid', 'invalid'])
2 diabetes dict_keys(['absent', 'present'])
3 sex dict_keys(['male', 'female'])
4  dict_keys([])
4 high_blood_pressure dict_keys(['absent', 'present'])
5  dict_keys([])
5  dict_keys([])
3 sex dict_keys(['female', 'male'])
4 high_blood_pressure dict_keys(['absent', 'present'])
5 anaemia dict_keys(['present', 'absent'])
6  dict_keys([])
6  dict_keys([])
5  dict_keys([])
4 high_blood_pressure dict_keys(['absent', 'present'])
5  dict_keys([])
5  dict_keys([])
2  dict_keys([])
1  dict_keys([])


In [ ]:
def compare(root, example):
  # a root with no name indicates it is leaf node
  if(root.name == ''):
    return root.classficationResult
  return compare(root.children[example[root.name]], example)

In [ ]:
def test(root, testingData):
  satisfyCnt = 0
  for idx, example in testingData.iterrows():
    result = (testingData.iloc[0]['DEATH_EVENT'] == 'yes')
    satisfyCnt += int(compare(root, example) == result)
  total = len(testingData)
  accuracy = (satisfyCnt/total)*100
  return accuracy

In [ ]:
accuracy = test(root, testingData)

In [ ]:
accuracy

72.28915662650603